# Graphillion Search for Psi_k

This notebook uses  to manage and search the space of triangle-free graphs on $ vertices.

We specifically search for graphs satisfying **Corollary 4** from the README, which equates property $\Psi_k$ (for  \le 3$) to:
1.  Every independent set of size $\le k$ has a common neighbor.
2.  The graph is twin-free ((x) \neq N(y)$).
3.  The graph is not isomorphic to a cycle {3m-1}$.

In [23]:
# Install graphillion if not present
!pip install graphillion

In [24]:
import networkx as nx
import matplotlib.pyplot as plt
import itertools
import sys
import pickle
import time
from graphillion import GraphSet


## Property Checkers

We implement the conditions of Corollary 4.

In [25]:
def is_independent_set(G, nodes):
    """Checks if the given set of nodes forms an Independent Set."""
    for u, v in itertools.combinations(nodes, 2):
        if G.has_edge(u, v):
            return False
    return True

def check_corollary_4_properties(G):
    """
    Checks if graph satisfies Corollary 4 properties:
    1. Every independent set of size <= 3 has a common neighbor.
    2. Twin-free (no two vertices have same neighbors).
    3. Not isomorphic to Andrasfai graph O_{3n-1}.
    """
    # Property 1: Common neighbor for IS of size <= 3
    nodes = list(G.nodes())
    for k in range(1, 4): # Sizes 1, 2, 3
        
        for subset in itertools.combinations(nodes, k):
            if is_independent_set(G, subset):
                # Check for common neighbor
                if not subset:
                    continue
                
                common_neighbors = set(G.neighbors(subset[0]))
                for node in subset[1:]:
                    common_neighbors &= set(G.neighbors(node))
                
                if not common_neighbors:
                    return False

    # Property 2: Twin-free
    for i in range(len(nodes)):
        for j in range(i + 1, len(nodes)):
            u, v = nodes[i], nodes[j]
            if set(G.neighbors(u)) == set(G.neighbors(v)):
                return False

    # Property 3: Not isomorphic to Andrasfai graph
    N = G.number_of_nodes()
    # Check if N is of form 3k - 1
    if (N + 1) % 3 == 0:
        # Construct Andrasfai graph of size N
        # Vertices 0..N-1
        # Edge (u, v) if (v-u) % N % 3 == 1
        A = nx.Graph()
        A.add_nodes_from(range(N))
        for u in range(N):
            for v in range(u + 1, N):
                diff = (v - u) % N
                if diff % 3 == 1:
                    A.add_edge(u, v)
        
        if nx.is_isomorphic(G, A):
            return False

    return True


## GraphSet Creation and Search

We use  to store the universe of triangle-free graphs and filter them.

In [ ]:
# Configuration
search_N = 9  # Graph size
search_k = 3   # Property Psi_k parameter

print(f"Setting up universe for N={search_N}...")
# Universe: All edges of K_N
universe_edges = list(nx.complete_graph(search_N).edges())
GraphSet.set_universe(universe_edges)

print(f"Generating triangle-free graphs of size {search_N} using Graphillion...")
start_gen = time.time()

# 1. Start with all possible subgraphs of the universe
gs = GraphSet.graphs()

# 2. Identify all triangles using cliques(3)
triangles = GraphSet.cliques(3)

# 3. Filter using forbidden_induced_subgraphs
# This removes any graph that contains any of the specified triangles as an induced subgraph
gs = gs.forbidden_induced_subgraphs(triangles)

# Note: Size calculation triggers the ZDD processing
gs_size = len(gs)
end_gen = time.time()
print(f"GraphSet created. Size: {gs_size}. Time elapsed: {end_gen - start_gen:.2f} seconds.")

print(f"Filtering for Psi_{search_k} (Corollary 4)...")
start_filter = time.time()
matches = []
count = 0

for edges in gs:
    count += 1
    # Reconstruct graph for property check
    G = nx.Graph()
    G.add_nodes_from(range(search_N))
    G.add_edges_from(edges)
    
    if check_corollary_4_properties(G):
        matches.append(G)
        
end_filter = time.time()
print(f"Found {len(matches)} graphs satisfying the property. Time elapsed: {end_filter - start_filter:.2f} seconds.")

Setting up universe for N=9...
Generating triangle-free graphs of size 9 using Graphillion...
GraphSet created. Size: 246348115. Time elapsed: 19.30 seconds.
Filtering for Psi_3 (Corollary 4)...


In [ ]:
# Save Results
filename = f"results_N{search_N}_k{search_k}.pkl"
with open(filename, "wb") as f:
    pickle.dump(matches, f)
print(f"Saved results to {filename}")

# Visualize first match
if matches:
    plt.figure(figsize=(8, 6))
    nx.draw(matches[0], with_labels=True, node_color="lightgreen", edge_color="black")
    plt.title(f"Example Graph Satisfying Psi_{search_k} (N={search_N})")
    plt.show()

Saved results to results_N8_k4.pkl
